In [13]:
import osmnx as ox
import folium
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union, polygonize
from concave_hull import concave_hull_indexes
import numpy as np

import osmnx as ox
import folium
import networkx as nx
from shapely.geometry import Point, MultiPoint
import alphashape

center_point = (48.8566, 2.3522)  # Coordonnées de Paris, France
speed_km_h = 15  # Vitesse en km/h
num_mins = 15  # Nombre de minutes
meters_per_minute = speed_km_h * 1000 / 60  # Conversion de la vitesse en mètres par minute

# Créer un graphe à partir du point central
G = ox.graph_from_point(center_point, dist=meters_per_minute * num_mins, network_type='drive')

# Trouver le nœud le plus proche du point central
nearest_node = ox.nearest_nodes(G, center_point[1], center_point[0])  # Notez l'ordre longitude, latitude

# Utiliser ce nœud comme centre pour ego_graph
subgraph = nx.ego_graph(G, nearest_node, radius=num_mins, distance='time')
isochrone = ox.graph_to_gdfs(G, edges=False)
isochrone_polygon = isochrone.unary_union.convex_hull


In [15]:
other_points = [(48.85,2.35), (48.86,2.35), (48.87,2.35)]
other_points_geom = [Point(lon, lat) for lat, lon in other_points]

valid_geometries = [geom for geom in other_points_geom if geom.is_valid]
points_within_isochrone = [point for point in valid_geometries if isochrone_polygon.contains(point)]

m = folium.Map(location=center_point, zoom_start=13)

#folium.GeoJson(Polygon(list(isochrone_polygon.exterior.coords))).add_to(m)

point_array = np.array([[point.x, point.y] for point in points])
idxes = concave_hull_indexes(
    point_array[:, :2],
    #length_threshold=1,
    concavity=0.8,
)

polygon_coords = point_array[idxes, ::-1].tolist()

folium.Polygon(locations=polygon_coords, color='blue', fill=True, fill_color='blue', fill_opacity=0.4).add_to(m)




# Adding markers for the center and points within the concave hull
folium.Marker(location=center_point, popup='Center Point', icon=folium.Icon(color='red')).add_to(m)
for point in points_within_isochrone:
    folium.Marker(location=(point.y, point.x), popup='Point Within 15 mins', icon=folium.Icon(color='green')).add_to(m)

m

In [5]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('my_map.png')

In [6]:
map_file_path = 'Downloads'
m.save(map_file_path)